In [1]:
!pip install -q datasets accelerate evaluate torch bitsandbytes 


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.4 MB/s eta 0:00:00:00:0100:01


In [2]:
import torch

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Huggingfacetoken")


In [4]:
from datasets import load_dataset

dataset = load_dataset("Malikeh1375/medical-question-answering-datasets", name="chatdoctor_icliniq")

print(dataset.column_names)
print(len(dataset['train']))
print(dataset["train"][:5])

dataset=dataset['train']

README.md:   0%|          | 0.00/4.96k [00:00<?, ?B/s]

(…)-00000-of-00001-bd1bc39748008404.parquet:   0%|          | 0.00/4.15M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7321 [00:00<?, ? examples/s]

{'train': ['instruction', 'input', 'output']}
7321
{'instruction': ['Answer this question truthfully', 'Answer this question truthfully', 'Answer this question truthfully', 'Answer this question truthfully', 'Answer this question truthfully'], 'input': ['Hello doctor,I had mumps five months ago and after that, I started to have an infection in my left testes. It was swollen and now it has shrunk to almost half the size of the other one. As I am sexually active, I feel a pain in each of the vas deferens after sex. If I do not have sex for days, they become sensitive. I was treated with Ceftum 500 mg, the first time I had an infection. Now my question is, is there any chance that the infection is still in my body? And, do I need to get examined for it? For the time being, please suggest some precautionary antibiotics for my relief.', 'Hello doctor,I am trying to conceive but my husband and I did cocaine a week ago. How long should my husband and I wait to safely continue to try to get pr

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id='openai-community/gpt2'
tokenizer= AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
# Tokenize the dataset
def tokenize_function(examples):
    combined_texts = [f"{ins} {i} {o}" for ins,i,o in zip(examples['instruction'],examples['input'], examples['output'])]
    tokenized = tokenizer(
        combined_texts,
        padding="max_length",
        truncation=True,
        max_length=512)
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])




Map:   0%|          | 0/7321 [00:00<?, ? examples/s]

In [7]:
!pip install peft
from peft import get_peft_model, LoraConfig, TaskType

In [8]:
bnb_config= BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
model=AutoModelForCausalLM.from_pretrained(model_id,quantization_config=bnb_config,torch_dtype=torch.bfloat16)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
from peft import get_peft_model , LoraConfig, TaskType, prepare_model_for_kbit_training
model=prepare_model_for_kbit_training(model)
lora_config=LoraConfig(
    r=16,
    lora_alpha=32,
    bias='none',
    lora_dropout=0.05,
    task_type='CAUSAL_LM'
)
model=get_peft_model(model,lora_config)


model.print_trainable_parameters()

trainable params: 589,824 || all params: 125,029,632 || trainable%: 0.4717


In [10]:
print(model)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2SdpaAttention(
              (c_attn): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=768, out_features=2304, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (l

In [11]:
import transformers
from datasets import load_dataset

model.config.use_cache = False
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="./gpt2_finetuned",
        report_to='none'
))

trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,10.699500
2,10.540500
3,10.525300
4,9.538800
5,10.319000
6,10.018700
7,10.494300
8,10.369100
9,10.404200
10,9.648500


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=1371, training_loss=1.7822467448674877, metrics={'train_runtime': 2004.7092, 'train_samples_per_second': 10.956, 'train_steps_per_second': 0.684, 'total_flos': 5773553546821632.0, 'train_loss': 1.7822467448674877, 'epoch': 2.998361551064992})

In [12]:
model.save_pretrained("./gpt2_finetuned_model")

In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

def load_model(model_path):
    config = PeftConfig.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
    model = PeftModel.from_pretrained(model, model_path)
    tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id
    return model, tokenizer

def generate_response(model, tokenizer, prompt, max_length=200):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.2,
            do_sample=True
            )
        
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


model_path = "./gpt2_finetuned_model"
model, tokenizer = load_model(model_path)

test_prompts = [
    "I have been experiencing frequent headaches and dizziness. Could this be a sign of a serious condition?",
    "What are the possible causes of chest pain other than a heart attack?",
    "How can I manage my high blood pressure naturally?",
    "Is it safe to take ibuprofen and paracetamol together for pain relief?",
    "What are the symptoms of vitamin D deficiency, and how can I treat it?"
]

for prompt in test_prompts:
    response = generate_response(model, tokenizer, prompt)
    print(f"User: {prompt}")
    print(f"Chatbot: {response}\n")
    print('-------END-------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: I have been experiencing frequent headaches and dizziness. Could this be a sign of a serious condition?
Chatbot: I have been experiencing frequent headaches and dizziness. Could this be a sign of a serious condition?
The symptoms are usually minor, but the headache is often very long lasting."

-------END-------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: What are the possible causes of chest pain other than a heart attack?
Chatbot: What are the possible causes of chest pain other than a heart attack?
A coronary artery bypass graft (CABG) is an artificial vascular structure that can be inserted into one or both lungs to block air flow. The Cabg has been shown in clinical trials and by medical experts as effective at preventing pulmonary embolism, lung cancer, cardiovascular disease, stroke/stroke etc. But it's still not clear whether this mechanism exists independently from atherosclerotic plaques such lesions found on arteries outside their circulatory systems like those inside veins called myocardium [1]. A recent study reported evidence for different effects within specific areas with increased risk factors associated specifically with type 2 diabetes mellitus including hyperglycemia[2]–(4). This means people who have low blood sugar but high glucose levels may experience metabolic syndrome which could lead to obesity due mainl

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: How can I manage my high blood pressure naturally?
Chatbot: How can I manage my high blood pressure naturally?
There are several ways to control your heart rate and keep it going. One is to use a pump that works at least twice as fast, or more often than not you will be able for some reason get back up after about an hour in the morning (usually during lunch time). This may happen by chance but usually they don't occur overnight so there isn`t much cause point nor effect between these two methods of controlling. Another method would involve using medication with no side effects whatsoever until one gets good enough sleep afterwards which then takes over 2 hours per night plus another 10 minutes before bedtime if sleeping well from 8am till midnight on daybreak since this means only half its normal schedule when all else fails either! You could also try taking low dose aspirin tablets while eating breakfast where we like them too because most people take almost nothing other than 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: Is it safe to take ibuprofen and paracetamol together for pain relief?
Chatbot: Is it safe to take ibuprofen and paracetamol together for pain relief?
Answer yes. Ibupserin is a relatively new drug that has been approved in Japan since 1995, but its efficacy remains limited because of the high cost involved in developing them (the typical dose used varies from one patient per day). For patients with chronic obstructive pulmonary disease or asthma who have not received adequate treatment due back surgery before they begin taking corticosteroids like acetaminophen/lactopramide tablets, then this may be their only option if there are no other options available after extensive clinical trials on long-term use as well; however you should also consider using any combination therapy including some selective serotonin reuptake inhibitors such Asperger's Syndrome antidepressants while still receiving all prescribed medication.[2] In addition these medications can cause side effects when t